In [1]:
# allow the notebook to access the parent directory so we can import the other modules
# https://stackoverflow.com/a/35273613
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

# Data Preparation
-----

### Constants and Folder Paths

In [2]:
import os
dataset_folder_path = os.path.join("..", "files", "dataset")
NUM_SAMPLES = 50

### Load Data and Split into *Test*, *Train/Valid*

In [3]:
from data.DataSet import DataSet
dataset = DataSet()
# dataset.load(dataset_folder_path, test_set_percentage=0.3333, validation_set_percentage=0.3333)
dataset.load(dataset_folder_path, test_set_percentage=0, validation_set_percentage=0)

In [4]:
print(len(dataset.train_data))
print(len(dataset.test_data))

3600
0


### Data Preprocessing

In [5]:
from utils.preprocessing import *
from functools import partial
dataset.apply(apply_mean_centering)
dataset.apply(apply_unit_distance_normalization)
#dataset.apply(partial(normalize_pressure_value, max_pressure_val=512))
dataset.apply(partial(spline_interpolate_and_resample, num_samples=NUM_SAMPLES))
dataset.expand(reverse_digit_sequence)
# dataset.apply(lambda digit: convert_xy_to_derivative(digit, normalize=False))
#dataset.apply(partial(convert_xy_to_derivative, normalize=True))

In [6]:
print(len(dataset.train_data))
print(len(dataset.test_data))

7200
0


### Setup Dataset, don't split, don't onehot encode, since we will perform cross validation

In [7]:
import numpy as np
from sklearn.model_selection import train_test_split

X_train = np.array(dataset.train_data)
# X_valid = np.array(dataset.valid_data)
# X_test = np.array(dataset.test_data)

Y_train = np.array(dataset.train_labels)
# Convert labels to numpy array and OneHot encode them
# encoder, Y_train, Y_valid, Y_test = dataset.onehot_encode_labels()
# Y_train = Y_train.astype('float32').todense()
# Y_valid = Y_valid.astype('float32').todense()
# Y_test = Y_test.astype('float32').todense()

In [8]:
Y_train.shape

(7200,)

# Test Constants

In [9]:
RANDOM_STATE = 42

----------
# **Naive GRU**
----------

In [10]:
PARAM_NUM_EPOCHS = 40
PARAM_BATCH_SIZE = 300

from models.naive_gru import NaiveGRU
from utils.evaluation import cross_validate_model

mymodel = NaiveGRU(X_train.shape[1:])
mymodel.batch_size = PARAM_BATCH_SIZE
mymodel.num_epochs = PARAM_NUM_EPOCHS

cross_validate_model(X_train, Y_train, mymodel, 3, RANDOM_STATE)

Using TensorFlow backend.



....................
Cross validation fold [1]
....................

Train on 4800 samples, validate on 2400 samples
Epoch 1/40
4800/4800 [==============================] - 2s 501us/step - loss: 2.1781 - categorical_accuracy: 0.1713 - val_loss: 2.1439 - val_categorical_accuracy: 0.2233
Epoch 2/40
4800/4800 [==============================] - 1s 287us/step - loss: 2.0399 - categorical_accuracy: 0.2396 - val_loss: 1.9179 - val_categorical_accuracy: 0.2792
Epoch 3/40
4800/4800 [==============================] - 1s 293us/step - loss: 1.5900 - categorical_accuracy: 0.4465 - val_loss: 1.3706 - val_categorical_accuracy: 0.5375
Epoch 4/40
4800/4800 [==============================] - 1s 288us/step - loss: 1.0067 - categorical_accuracy: 0.6848 - val_loss: 0.9128 - val_categorical_accuracy: 0.7337
Epoch 5/40
4800/4800 [==============================] - 1s 290us/step - loss: 0.5976 - categorical_accuracy: 0.8179 - val_loss: 0.5906 - val_categorical_accuracy: 0.8158
Epoch 6/40
4800/4800 [==========

4800/4800 [==============================] - 1s 288us/step - loss: 0.2307 - categorical_accuracy: 0.9390 - val_loss: 0.3172 - val_categorical_accuracy: 0.9154
Epoch 8/40
4800/4800 [==============================] - 1s 284us/step - loss: 0.1645 - categorical_accuracy: 0.9573 - val_loss: 0.3127 - val_categorical_accuracy: 0.9187
Epoch 9/40
4800/4800 [==============================] - 1s 288us/step - loss: 0.1222 - categorical_accuracy: 0.9729 - val_loss: 0.2624 - val_categorical_accuracy: 0.9308
Epoch 10/40
4800/4800 [==============================] - 1s 283us/step - loss: 0.0960 - categorical_accuracy: 0.9775 - val_loss: 0.2644 - val_categorical_accuracy: 0.9275
Epoch 11/40
4800/4800 [==============================] - 1s 284us/step - loss: 0.0863 - categorical_accuracy: 0.9792 - val_loss: 0.2630 - val_categorical_accuracy: 0.9292
Epoch 12/40
4800/4800 [==============================] - 1s 285us/step - loss: 0.0934 - categorical_accuracy: 0.9744 - val_loss: 0.2436 - val_categorical_accur

Epoch 14/40
4800/4800 [==============================] - 2s 327us/step - loss: 0.0722 - categorical_accuracy: 0.9823 - val_loss: 0.3194 - val_categorical_accuracy: 0.9204
Epoch 15/40
4800/4800 [==============================] - 1s 305us/step - loss: 0.0733 - categorical_accuracy: 0.9808 - val_loss: 0.3285 - val_categorical_accuracy: 0.9188
Epoch 16/40
4800/4800 [==============================] - 1s 281us/step - loss: 0.0588 - categorical_accuracy: 0.9840 - val_loss: 0.3340 - val_categorical_accuracy: 0.9146
Epoch 17/40
4800/4800 [==============================] - 1s 278us/step - loss: 0.0557 - categorical_accuracy: 0.9838 - val_loss: 0.3194 - val_categorical_accuracy: 0.9229
Epoch 18/40
4800/4800 [==============================] - 1s 280us/step - loss: 0.0601 - categorical_accuracy: 0.9842 - val_loss: 0.3460 - val_categorical_accuracy: 0.9121
Epoch 19/40
4800/4800 [==============================] - 1s 278us/step - loss: 0.0512 - categorical_accuracy: 0.9850 - val_loss: 0.3034 - val_cat

[93.625, 94.666666666666671, 93.541666666666671]

----------
# **Regularized Deep GRU**
----------

#### Batch Size 300

In [11]:
PARAM_NUM_EPOCHS = 40
PARAM_BATCH_SIZE = 300

from models.regularized_deep_gru import RegularizedDeepGRU
from utils.evaluation import cross_validate_model

mymodel = RegularizedDeepGRU(X_train.shape[1:])
mymodel.batch_size = PARAM_BATCH_SIZE
mymodel.num_epochs = PARAM_NUM_EPOCHS

cross_validate_model(X_train, Y_train, mymodel, 3, RANDOM_STATE)

Using TensorFlow backend.



....................
Cross validation fold [1]
....................

Train on 4800 samples, validate on 2400 samples
Epoch 1/40
4800/4800 [==============================] - 4s 897us/step - loss: 2.1520 - categorical_accuracy: 0.2025 - val_loss: 2.0329 - val_categorical_accuracy: 0.2250
Epoch 2/40
4800/4800 [==============================] - 3s 677us/step - loss: 1.7538 - categorical_accuracy: 0.3471 - val_loss: 1.5116 - val_categorical_accuracy: 0.4921
Epoch 3/40
4800/4800 [==============================] - 3s 679us/step - loss: 1.2495 - categorical_accuracy: 0.5623 - val_loss: 1.1988 - val_categorical_accuracy: 0.6183
Epoch 4/40
4800/4800 [==============================] - 3s 678us/step - loss: 0.8755 - categorical_accuracy: 0.7067 - val_loss: 0.7724 - val_categorical_accuracy: 0.7738
Epoch 5/40
4800/4800 [==============================] - 3s 680us/step - loss: 0.5751 - categorical_accuracy: 0.8146 - val_loss: 0.6025 - val_categorical_accuracy: 0.8279
Epoch 6/40
4800/4800 [==========

Epoch 14/40
4800/4800 [==============================] - 3s 673us/step - loss: 0.0802 - categorical_accuracy: 0.9806 - val_loss: 0.3810 - val_categorical_accuracy: 0.9037
Epoch 15/40
4800/4800 [==============================] - 3s 670us/step - loss: 0.0784 - categorical_accuracy: 0.9788 - val_loss: 0.3723 - val_categorical_accuracy: 0.9083
Epoch 16/40
4800/4800 [==============================] - 3s 671us/step - loss: 0.0638 - categorical_accuracy: 0.9844 - val_loss: 0.3785 - val_categorical_accuracy: 0.9050
Epoch 17/40
4800/4800 [==============================] - 3s 671us/step - loss: 0.0681 - categorical_accuracy: 0.9835 - val_loss: 0.3618 - val_categorical_accuracy: 0.9117
Epoch 18/40
4800/4800 [==============================] - 3s 673us/step - loss: 0.0632 - categorical_accuracy: 0.9817 - val_loss: 0.3760 - val_categorical_accuracy: 0.9108
Epoch 19/40
4800/4800 [==============================] - 3s 697us/step - loss: 0.0526 - categorical_accuracy: 0.9869 - val_loss: 0.4071 - val_cat

[95.0, 95.25, 91.25]

In [12]:
PARAM_NUM_EPOCHS = 80
PARAM_BATCH_SIZE = 300

from models.regularized_deep_gru import RegularizedDeepGRU
from utils.evaluation import cross_validate_model

mymodel = RegularizedDeepGRU(X_train.shape[1:])
mymodel.batch_size = PARAM_BATCH_SIZE
mymodel.num_epochs = PARAM_NUM_EPOCHS

cross_validate_model(X_train, Y_train, mymodel, 3, RANDOM_STATE)


....................
Cross validation fold [1]
....................

Train on 4800 samples, validate on 2400 samples
Epoch 1/80
4800/4800 [==============================] - 6s 1ms/step - loss: 2.1299 - categorical_accuracy: 0.2198 - val_loss: 2.0047 - val_categorical_accuracy: 0.2367
Epoch 2/80
4800/4800 [==============================] - 3s 682us/step - loss: 1.6836 - categorical_accuracy: 0.3779 - val_loss: 1.4554 - val_categorical_accuracy: 0.4933
Epoch 3/80
4800/4800 [==============================] - 3s 681us/step - loss: 1.2066 - categorical_accuracy: 0.5752 - val_loss: 1.0793 - val_categorical_accuracy: 0.6725
Epoch 4/80
4800/4800 [==============================] - 3s 681us/step - loss: 0.9093 - categorical_accuracy: 0.7040 - val_loss: 0.8045 - val_categorical_accuracy: 0.7513
Epoch 5/80
4800/4800 [==============================] - 3s 699us/step - loss: 0.5985 - categorical_accuracy: 0.8188 - val_loss: 0.5617 - val_categorical_accuracy: 0.8321
Epoch 6/80
4800/4800 [============

Epoch 15/80
4800/4800 [==============================] - 3s 693us/step - loss: 0.0845 - categorical_accuracy: 0.9754 - val_loss: 0.2450 - val_categorical_accuracy: 0.9337
Epoch 16/80
4800/4800 [==============================] - 3s 688us/step - loss: 0.0697 - categorical_accuracy: 0.9804 - val_loss: 0.2184 - val_categorical_accuracy: 0.9396
Epoch 17/80
4800/4800 [==============================] - 3s 692us/step - loss: 0.0682 - categorical_accuracy: 0.9815 - val_loss: 0.2283 - val_categorical_accuracy: 0.9433
Epoch 18/80
4800/4800 [==============================] - 3s 684us/step - loss: 0.0561 - categorical_accuracy: 0.9846 - val_loss: 0.2317 - val_categorical_accuracy: 0.9412
Epoch 19/80
4800/4800 [==============================] - 3s 684us/step - loss: 0.0498 - categorical_accuracy: 0.9871 - val_loss: 0.2414 - val_categorical_accuracy: 0.9400
Epoch 20/80
4800/4800 [==============================] - 3s 686us/step - loss: 0.0503 - categorical_accuracy: 0.9865 - val_loss: 0.2108 - val_cat

Epoch 63/80
4800/4800 [==============================] - 3s 681us/step - loss: 0.0043 - categorical_accuracy: 0.9994 - val_loss: 0.2218 - val_categorical_accuracy: 0.9554
Epoch 64/80
4800/4800 [==============================] - 3s 681us/step - loss: 0.0039 - categorical_accuracy: 0.9994 - val_loss: 0.2445 - val_categorical_accuracy: 0.9529
Epoch 65/80
4800/4800 [==============================] - 3s 692us/step - loss: 0.0026 - categorical_accuracy: 0.9998 - val_loss: 0.2267 - val_categorical_accuracy: 0.9554
Epoch 66/80
4800/4800 [==============================] - 3s 687us/step - loss: 0.0025 - categorical_accuracy: 0.9996 - val_loss: 0.2140 - val_categorical_accuracy: 0.9600
Epoch 67/80
4800/4800 [==============================] - 3s 683us/step - loss: 0.0026 - categorical_accuracy: 0.9996 - val_loss: 0.2248 - val_categorical_accuracy: 0.9592
Epoch 68/80
4800/4800 [==============================] - 3s 679us/step - loss: 0.0032 - categorical_accuracy: 0.9990 - val_loss: 0.2201 - val_cat

Epoch 30/80
4800/4800 [==============================] - 3s 680us/step - loss: 0.0285 - categorical_accuracy: 0.9929 - val_loss: 0.2969 - val_categorical_accuracy: 0.9371
Epoch 31/80
4800/4800 [==============================] - 3s 681us/step - loss: 0.0264 - categorical_accuracy: 0.9921 - val_loss: 0.3174 - val_categorical_accuracy: 0.9288
Epoch 32/80
4800/4800 [==============================] - 3s 680us/step - loss: 0.0214 - categorical_accuracy: 0.9944 - val_loss: 0.2884 - val_categorical_accuracy: 0.9425
Epoch 33/80
4800/4800 [==============================] - 4s 736us/step - loss: 0.0232 - categorical_accuracy: 0.9929 - val_loss: 0.2945 - val_categorical_accuracy: 0.9400
Epoch 34/80
4800/4800 [==============================] - 3s 707us/step - loss: 0.0309 - categorical_accuracy: 0.9902 - val_loss: 0.3074 - val_categorical_accuracy: 0.9354
Epoch 35/80
4800/4800 [==============================] - 3s 686us/step - loss: 0.0281 - categorical_accuracy: 0.9923 - val_loss: 0.3101 - val_cat

Epoch 78/80
4800/4800 [==============================] - 3s 678us/step - loss: 0.0072 - categorical_accuracy: 0.9979 - val_loss: 0.3177 - val_categorical_accuracy: 0.9454
Epoch 79/80
4800/4800 [==============================] - 3s 680us/step - loss: 0.0119 - categorical_accuracy: 0.9967 - val_loss: 0.3323 - val_categorical_accuracy: 0.9488
Epoch 80/80
2400/2400 [==============================] - 1s 495us/step
categorical_accuracy: 94.92%
94.57% (+/- 0.84%)


[93.416666666666671, 95.375, 94.916666666666671]

#### Batch Size 600

In [12]:
PARAM_NUM_EPOCHS = 40
PARAM_BATCH_SIZE = 600

from models.regularized_deep_gru import RegularizedDeepGRU
from utils.evaluation import cross_validate_model

mymodel = RegularizedDeepGRU(X_train.shape[1:])
mymodel.batch_size = PARAM_BATCH_SIZE
mymodel.num_epochs = PARAM_NUM_EPOCHS

cross_validate_model(X_train, Y_train, mymodel, 3, RANDOM_STATE)


....................
Cross validation fold [1]
....................

Train on 4800 samples, validate on 2400 samples
Epoch 1/40
4800/4800 [==============================] - 4s 766us/step - loss: 2.2194 - categorical_accuracy: 0.1829 - val_loss: 2.1009 - val_categorical_accuracy: 0.2496
Epoch 2/40
4800/4800 [==============================] - 3s 582us/step - loss: 2.0138 - categorical_accuracy: 0.2492 - val_loss: 1.8176 - val_categorical_accuracy: 0.3242
Epoch 3/40
4800/4800 [==============================] - 3s 563us/step - loss: 1.6939 - categorical_accuracy: 0.3723 - val_loss: 1.5784 - val_categorical_accuracy: 0.4204
Epoch 4/40
4800/4800 [==============================] - 3s 615us/step - loss: 1.4508 - categorical_accuracy: 0.4773 - val_loss: 1.3909 - val_categorical_accuracy: 0.4958
Epoch 5/40
4800/4800 [==============================] - 3s 571us/step - loss: 1.1906 - categorical_accuracy: 0.5783 - val_loss: 1.0936 - val_categorical_accuracy: 0.6638
Epoch 6/40
4800/4800 [==========

Epoch 14/40
4800/4800 [==============================] - 3s 575us/step - loss: 0.2537 - categorical_accuracy: 0.9242 - val_loss: 0.3456 - val_categorical_accuracy: 0.9125
Epoch 15/40
4800/4800 [==============================] - 3s 601us/step - loss: 0.1846 - categorical_accuracy: 0.9500 - val_loss: 0.3338 - val_categorical_accuracy: 0.9183
Epoch 16/40
4800/4800 [==============================] - 3s 571us/step - loss: 0.1661 - categorical_accuracy: 0.9546 - val_loss: 0.3277 - val_categorical_accuracy: 0.9154
Epoch 17/40
4800/4800 [==============================] - 3s 623us/step - loss: 0.1723 - categorical_accuracy: 0.9502 - val_loss: 0.3552 - val_categorical_accuracy: 0.9096
Epoch 18/40
4800/4800 [==============================] - 3s 579us/step - loss: 0.1392 - categorical_accuracy: 0.9617 - val_loss: 0.3060 - val_categorical_accuracy: 0.9292
Epoch 19/40
4800/4800 [==============================] - 3s 575us/step - loss: 0.1249 - categorical_accuracy: 0.9652 - val_loss: 0.3058 - val_cat

[93.208333333333343, 91.708333333333343, 94.208333333333343]

----------
# **Regularized Deep LSTM**
----------

#### 40 epochs

In [14]:
PARAM_NUM_EPOCHS = 40
PARAM_BATCH_SIZE = 300

from models.regularized_deep_lstm import RegularizedDeepLSTM
from utils.evaluation import cross_validate_model

mymodel = RegularizedDeepLSTM(X_train.shape[1:])
mymodel.batch_size = PARAM_BATCH_SIZE
mymodel.num_epochs = PARAM_NUM_EPOCHS

cross_validate_model(X_train, Y_train, mymodel, 3, RANDOM_STATE)


....................
Cross validation fold [1]
....................

Train on 4800 samples, validate on 2400 samples
Epoch 1/40
4800/4800 [==============================] - 6s 1ms/step - loss: 2.2044 - categorical_accuracy: 0.1696 - val_loss: 2.1850 - val_categorical_accuracy: 0.1663
Epoch 2/40
4800/4800 [==============================] - 4s 932us/step - loss: 1.9835 - categorical_accuracy: 0.2535 - val_loss: 1.6022 - val_categorical_accuracy: 0.5400
Epoch 3/40
4800/4800 [==============================] - 4s 930us/step - loss: 1.8053 - categorical_accuracy: 0.3365 - val_loss: 1.2549 - val_categorical_accuracy: 0.6200
Epoch 4/40
4800/4800 [==============================] - 5s 950us/step - loss: 1.6462 - categorical_accuracy: 0.4052 - val_loss: 1.4229 - val_categorical_accuracy: 0.5092
Epoch 5/40
4800/4800 [==============================] - 5s 939us/step - loss: 1.5111 - categorical_accuracy: 0.4552 - val_loss: 1.1066 - val_categorical_accuracy: 0.6225
Epoch 6/40
4800/4800 [============

Epoch 14/40
4800/4800 [==============================] - 4s 928us/step - loss: 0.5595 - categorical_accuracy: 0.8094 - val_loss: 0.5887 - val_categorical_accuracy: 0.8167
Epoch 15/40
4800/4800 [==============================] - 4s 927us/step - loss: 0.4965 - categorical_accuracy: 0.8273 - val_loss: 0.4890 - val_categorical_accuracy: 0.8496
Epoch 16/40
4800/4800 [==============================] - 4s 926us/step - loss: 0.4551 - categorical_accuracy: 0.8448 - val_loss: 0.4614 - val_categorical_accuracy: 0.8704
Epoch 17/40
4800/4800 [==============================] - 4s 932us/step - loss: 0.4572 - categorical_accuracy: 0.8446 - val_loss: 0.4879 - val_categorical_accuracy: 0.8575
Epoch 18/40
4800/4800 [==============================] - 4s 927us/step - loss: 0.3808 - categorical_accuracy: 0.8723 - val_loss: 0.4115 - val_categorical_accuracy: 0.8746
Epoch 19/40
4800/4800 [==============================] - 4s 929us/step - loss: 0.3583 - categorical_accuracy: 0.8815 - val_loss: 0.4010 - val_cat

[95.75, 95.458333333333329, 92.958333333333329]

#### 80 epochs

In [11]:
PARAM_NUM_EPOCHS = 80
PARAM_BATCH_SIZE = 300

from models.regularized_deep_lstm import RegularizedDeepLSTM
from utils.evaluation import cross_validate_model

mymodel = RegularizedDeepLSTM(X_train.shape[1:])
mymodel.batch_size = PARAM_BATCH_SIZE
mymodel.num_epochs = PARAM_NUM_EPOCHS

cross_validate_model(X_train, Y_train, mymodel, 3, RANDOM_STATE)


....................
Cross validation fold [1]
....................

Train on 4800 samples, validate on 2400 samples
Epoch 1/80
4800/4800 [==============================] - 7s 1ms/step - loss: 2.2126 - categorical_accuracy: 0.1658 - val_loss: 2.1815 - val_categorical_accuracy: 0.1350
Epoch 2/80
4800/4800 [==============================] - 5s 947us/step - loss: 1.9922 - categorical_accuracy: 0.2469 - val_loss: 1.7353 - val_categorical_accuracy: 0.4437
Epoch 3/80
4800/4800 [==============================] - 5s 945us/step - loss: 1.8640 - categorical_accuracy: 0.3052 - val_loss: 1.5108 - val_categorical_accuracy: 0.5304
Epoch 4/80
4800/4800 [==============================] - 5s 942us/step - loss: 1.7202 - categorical_accuracy: 0.3742 - val_loss: 1.4862 - val_categorical_accuracy: 0.4996
Epoch 5/80
4800/4800 [==============================] - 5s 942us/step - loss: 1.5587 - categorical_accuracy: 0.4302 - val_loss: 1.4051 - val_categorical_accuracy: 0.5021
Epoch 6/80
4800/4800 [============

Epoch 15/80
4800/4800 [==============================] - 5s 965us/step - loss: 0.5814 - categorical_accuracy: 0.7948 - val_loss: 0.5798 - val_categorical_accuracy: 0.8117
Epoch 16/80
4800/4800 [==============================] - 5s 981us/step - loss: 0.5449 - categorical_accuracy: 0.8102 - val_loss: 0.4447 - val_categorical_accuracy: 0.8596
Epoch 17/80
4800/4800 [==============================] - 5s 1ms/step - loss: 0.5332 - categorical_accuracy: 0.8104 - val_loss: 0.4578 - val_categorical_accuracy: 0.8696
Epoch 18/80
4800/4800 [==============================] - 5s 1ms/step - loss: 0.4733 - categorical_accuracy: 0.8394 - val_loss: 0.3463 - val_categorical_accuracy: 0.9017
Epoch 19/80
4800/4800 [==============================] - 5s 980us/step - loss: 0.4276 - categorical_accuracy: 0.8565 - val_loss: 0.3565 - val_categorical_accuracy: 0.8900
Epoch 20/80
4800/4800 [==============================] - 5s 1ms/step - loss: 0.4115 - categorical_accuracy: 0.8583 - val_loss: 0.3439 - val_categoric

Epoch 63/80
4800/4800 [==============================] - 4s 936us/step - loss: 0.1133 - categorical_accuracy: 0.9602 - val_loss: 0.1891 - val_categorical_accuracy: 0.9558
Epoch 64/80
4800/4800 [==============================] - 5s 960us/step - loss: 0.1156 - categorical_accuracy: 0.9600 - val_loss: 0.2023 - val_categorical_accuracy: 0.9538
Epoch 65/80
4800/4800 [==============================] - 5s 1ms/step - loss: 0.1153 - categorical_accuracy: 0.9644 - val_loss: 0.2359 - val_categorical_accuracy: 0.9454
Epoch 66/80
4800/4800 [==============================] - 5s 945us/step - loss: 0.1338 - categorical_accuracy: 0.9554 - val_loss: 0.2188 - val_categorical_accuracy: 0.9492
Epoch 67/80
4800/4800 [==============================] - 5s 947us/step - loss: 0.1147 - categorical_accuracy: 0.9627 - val_loss: 0.1705 - val_categorical_accuracy: 0.9587
Epoch 68/80
4800/4800 [==============================] - 5s 951us/step - loss: 0.1008 - categorical_accuracy: 0.9658 - val_loss: 0.2083 - val_categ

Epoch 30/80
4800/4800 [==============================] - 5s 949us/step - loss: 0.2226 - categorical_accuracy: 0.9275 - val_loss: 0.2863 - val_categorical_accuracy: 0.9225
Epoch 31/80
4800/4800 [==============================] - 5s 1ms/step - loss: 0.2293 - categorical_accuracy: 0.9273 - val_loss: 0.3160 - val_categorical_accuracy: 0.9183
Epoch 32/80
4800/4800 [==============================] - 5s 941us/step - loss: 0.2055 - categorical_accuracy: 0.9340 - val_loss: 0.3149 - val_categorical_accuracy: 0.9262
Epoch 33/80
4800/4800 [==============================] - 5s 1ms/step - loss: 0.2287 - categorical_accuracy: 0.9267 - val_loss: 0.3323 - val_categorical_accuracy: 0.9167
Epoch 34/80
4800/4800 [==============================] - 5s 1ms/step - loss: 0.2179 - categorical_accuracy: 0.9294 - val_loss: 0.3056 - val_categorical_accuracy: 0.9271
Epoch 35/80
4800/4800 [==============================] - 5s 953us/step - loss: 0.2085 - categorical_accuracy: 0.9317 - val_loss: 0.2993 - val_categoric

Epoch 78/80
4800/4800 [==============================] - 4s 935us/step - loss: 0.1041 - categorical_accuracy: 0.9625 - val_loss: 0.3048 - val_categorical_accuracy: 0.9321
Epoch 79/80
4800/4800 [==============================] - 4s 936us/step - loss: 0.1076 - categorical_accuracy: 0.9623 - val_loss: 0.2941 - val_categorical_accuracy: 0.9329
Epoch 80/80
2400/2400 [==============================] - 2s 712us/step
categorical_accuracy: 92.46%
94.14% (+/- 1.53%)


[93.791666666666657, 96.166666666666671, 92.458333333333329]

----------
# **Regularized Deep Bidirectional LSTM**
----------

In [10]:
PARAM_NUM_EPOCHS = 40
PARAM_BATCH_SIZE = 300

from models.regularized_deep_bidirectional_lstm import RegularizedDeepBidirectionalLSTM
from utils.evaluation import cross_validate_model

mymodel = RegularizedDeepBidirectionalLSTM(X_train.shape[1:])
mymodel.batch_size = PARAM_BATCH_SIZE
mymodel.num_epochs = PARAM_NUM_EPOCHS

cross_validate_model(X_train, Y_train, mymodel, 3, RANDOM_STATE)

Using TensorFlow backend.



....................
Cross validation fold [1]
....................

Train on 4800 samples, validate on 2400 samples
Epoch 1/40
4800/4800 [==============================] - 13s 3ms/step - loss: 2.0170 - categorical_accuracy: 0.2960 - val_loss: 1.6091 - val_categorical_accuracy: 0.4342
Epoch 2/40
4800/4800 [==============================] - 11s 2ms/step - loss: 1.4302 - categorical_accuracy: 0.5081 - val_loss: 0.9185 - val_categorical_accuracy: 0.7138
Epoch 3/40
4800/4800 [==============================] - 11s 2ms/step - loss: 1.1046 - categorical_accuracy: 0.6154 - val_loss: 0.6364 - val_categorical_accuracy: 0.8079
Epoch 4/40
4800/4800 [==============================] - 11s 2ms/step - loss: 0.8992 - categorical_accuracy: 0.6919 - val_loss: 0.4862 - val_categorical_accuracy: 0.8396
Epoch 5/40
4800/4800 [==============================] - 11s 2ms/step - loss: 0.8101 - categorical_accuracy: 0.7140 - val_loss: 0.4425 - val_categorical_accuracy: 0.8704
Epoch 6/40
4800/4800 [===============

4800/4800 [==============================] - 11s 2ms/step - loss: 0.2544 - categorical_accuracy: 0.9183 - val_loss: 0.3818 - val_categorical_accuracy: 0.9121
Epoch 15/40
4800/4800 [==============================] - 11s 2ms/step - loss: 0.2085 - categorical_accuracy: 0.9329 - val_loss: 0.3249 - val_categorical_accuracy: 0.9167
Epoch 16/40
4800/4800 [==============================] - 11s 2ms/step - loss: 0.2006 - categorical_accuracy: 0.9387 - val_loss: 0.3005 - val_categorical_accuracy: 0.9271
Epoch 17/40
4800/4800 [==============================] - 11s 2ms/step - loss: 0.1797 - categorical_accuracy: 0.9421 - val_loss: 0.2740 - val_categorical_accuracy: 0.9258
Epoch 18/40
4800/4800 [==============================] - 11s 2ms/step - loss: 0.1571 - categorical_accuracy: 0.9515 - val_loss: 0.3497 - val_categorical_accuracy: 0.9212
Epoch 19/40
4800/4800 [==============================] - 11s 2ms/step - loss: 0.1499 - categorical_accuracy: 0.9521 - val_loss: 0.3231 - val_categorical_accuracy:

[93.916666666666671, 96.5, 93.75]

----------
# **Regularized 1024 GRU**
----------

In [14]:
PARAM_NUM_EPOCHS = 20
PARAM_BATCH_SIZE = 300

from models.regularized_1024_gru import Regularized1024GRU
from utils.evaluation import cross_validate_model

mymodel = Regularized1024GRU(X_train.shape[1:])
mymodel.batch_size = PARAM_BATCH_SIZE
mymodel.num_epochs = PARAM_NUM_EPOCHS

cross_validate_model(X_train, Y_train, mymodel, 3, RANDOM_STATE)


....................
Cross validation fold [1]
....................

Train on 4800 samples, validate on 2400 samples
Epoch 1/20
4800/4800 [==============================] - 13s 3ms/step - loss: 2.0314 - categorical_accuracy: 0.2331 - val_loss: 1.7715 - val_categorical_accuracy: 0.3362
Epoch 2/20
4800/4800 [==============================] - 10s 2ms/step - loss: 1.5816 - categorical_accuracy: 0.4110 - val_loss: 1.5434 - val_categorical_accuracy: 0.4421
Epoch 3/20
4800/4800 [==============================] - 10s 2ms/step - loss: 1.1800 - categorical_accuracy: 0.5771 - val_loss: 1.2643 - val_categorical_accuracy: 0.5629
Epoch 4/20
4800/4800 [==============================] - 10s 2ms/step - loss: 0.8433 - categorical_accuracy: 0.7142 - val_loss: 0.7328 - val_categorical_accuracy: 0.7733
Epoch 5/20
4800/4800 [==============================] - 10s 2ms/step - loss: 0.5726 - categorical_accuracy: 0.8144 - val_loss: 0.5363 - val_categorical_accuracy: 0.8283
Epoch 6/20
4800/4800 [===============

[94.083333333333329, 95.125, 93.833333333333329]

----------
# **Regularized 64 GRU**
----------

In [10]:
PARAM_NUM_EPOCHS = 80
PARAM_BATCH_SIZE = 300

from models.regularized_64_gru import Regularized64GRU
from utils.evaluation import cross_validate_model

mymodel = Regularized64GRU(X_train.shape[1:])
mymodel.batch_size = PARAM_BATCH_SIZE
mymodel.num_epochs = PARAM_NUM_EPOCHS

cross_validate_model(X_train, Y_train, mymodel, 3, RANDOM_STATE)

Using TensorFlow backend.



....................
Cross validation fold [1]
....................

Train on 4800 samples, validate on 2400 samples
Epoch 1/80
4800/4800 [==============================] - 2s 482us/step - loss: 2.2321 - categorical_accuracy: 0.1702 - val_loss: 2.1181 - val_categorical_accuracy: 0.2642
Epoch 2/80
4800/4800 [==============================] - 1s 255us/step - loss: 2.0600 - categorical_accuracy: 0.2304 - val_loss: 1.8773 - val_categorical_accuracy: 0.3250
Epoch 3/80
4800/4800 [==============================] - 1s 254us/step - loss: 1.8150 - categorical_accuracy: 0.3379 - val_loss: 1.5542 - val_categorical_accuracy: 0.4450
Epoch 4/80
4800/4800 [==============================] - 1s 256us/step - loss: 1.5284 - categorical_accuracy: 0.4456 - val_loss: 1.3952 - val_categorical_accuracy: 0.5558
Epoch 5/80
4800/4800 [==============================] - 1s 257us/step - loss: 1.2856 - categorical_accuracy: 0.5446 - val_loss: 1.1637 - val_categorical_accuracy: 0.6150
Epoch 6/80
4800/4800 [==========

Epoch 15/80
4800/4800 [==============================] - 1s 269us/step - loss: 0.2851 - categorical_accuracy: 0.9183 - val_loss: 0.4024 - val_categorical_accuracy: 0.9054
Epoch 16/80
4800/4800 [==============================] - 1s 258us/step - loss: 0.2470 - categorical_accuracy: 0.9267 - val_loss: 0.3805 - val_categorical_accuracy: 0.9154
Epoch 17/80
4800/4800 [==============================] - 1s 260us/step - loss: 0.2459 - categorical_accuracy: 0.9313 - val_loss: 0.3795 - val_categorical_accuracy: 0.9112
Epoch 18/80
4800/4800 [==============================] - 1s 260us/step - loss: 0.2250 - categorical_accuracy: 0.9365 - val_loss: 0.3657 - val_categorical_accuracy: 0.9233
Epoch 19/80
4800/4800 [==============================] - 1s 256us/step - loss: 0.1984 - categorical_accuracy: 0.9442 - val_loss: 0.3640 - val_categorical_accuracy: 0.9150
Epoch 20/80
4800/4800 [==============================] - 1s 257us/step - loss: 0.2105 - categorical_accuracy: 0.9421 - val_loss: 0.3678 - val_cat

Epoch 63/80
4800/4800 [==============================] - 1s 257us/step - loss: 0.0427 - categorical_accuracy: 0.9877 - val_loss: 0.4173 - val_categorical_accuracy: 0.9275
Epoch 64/80
4800/4800 [==============================] - 1s 258us/step - loss: 0.0395 - categorical_accuracy: 0.9900 - val_loss: 0.3464 - val_categorical_accuracy: 0.9408
Epoch 65/80
4800/4800 [==============================] - 1s 260us/step - loss: 0.0407 - categorical_accuracy: 0.9894 - val_loss: 0.3435 - val_categorical_accuracy: 0.9412
Epoch 66/80
4800/4800 [==============================] - 1s 264us/step - loss: 0.0307 - categorical_accuracy: 0.9919 - val_loss: 0.3902 - val_categorical_accuracy: 0.9287
Epoch 67/80
4800/4800 [==============================] - 1s 257us/step - loss: 0.0326 - categorical_accuracy: 0.9913 - val_loss: 0.3549 - val_categorical_accuracy: 0.9412
Epoch 68/80
4800/4800 [==============================] - 1s 261us/step - loss: 0.0350 - categorical_accuracy: 0.9892 - val_loss: 0.3832 - val_cat

Epoch 30/80
4800/4800 [==============================] - 1s 257us/step - loss: 0.1159 - categorical_accuracy: 0.9723 - val_loss: 0.2328 - val_categorical_accuracy: 0.9508
Epoch 31/80
4800/4800 [==============================] - 1s 254us/step - loss: 0.1197 - categorical_accuracy: 0.9717 - val_loss: 0.2369 - val_categorical_accuracy: 0.9450
Epoch 32/80
4800/4800 [==============================] - 1s 256us/step - loss: 0.1149 - categorical_accuracy: 0.9754 - val_loss: 0.2220 - val_categorical_accuracy: 0.9467
Epoch 33/80
4800/4800 [==============================] - 1s 257us/step - loss: 0.1145 - categorical_accuracy: 0.9713 - val_loss: 0.2478 - val_categorical_accuracy: 0.9454
Epoch 34/80
4800/4800 [==============================] - 1s 255us/step - loss: 0.1069 - categorical_accuracy: 0.9740 - val_loss: 0.2734 - val_categorical_accuracy: 0.9367
Epoch 35/80
4800/4800 [==============================] - 1s 256us/step - loss: 0.0896 - categorical_accuracy: 0.9781 - val_loss: 0.2890 - val_cat

Epoch 78/80
4800/4800 [==============================] - 1s 258us/step - loss: 0.0445 - categorical_accuracy: 0.9896 - val_loss: 0.3201 - val_categorical_accuracy: 0.9404
Epoch 79/80
4800/4800 [==============================] - 1s 257us/step - loss: 0.0380 - categorical_accuracy: 0.9898 - val_loss: 0.3011 - val_categorical_accuracy: 0.9454
Epoch 80/80
2400/2400 [==============================] - 1s 487us/step
categorical_accuracy: 95.29%
94.19% (+/- 0.85%)


[94.083333333333329, 93.208333333333343, 95.291666666666657]

----------
# **Regularized 4x64 GRU**
----------

In [11]:
PARAM_NUM_EPOCHS = 40
PARAM_BATCH_SIZE = 300

from models.regularized_4x64_gru import Regularized4x64GRU
from utils.evaluation import cross_validate_model

mymodel = Regularized4x64GRU(X_train.shape[1:])
mymodel.batch_size = PARAM_BATCH_SIZE
mymodel.num_epochs = PARAM_NUM_EPOCHS

cross_validate_model(X_train, Y_train, mymodel, 3, RANDOM_STATE)


....................
Cross validation fold [1]
....................

Train on 4800 samples, validate on 2400 samples
Epoch 1/40
4800/4800 [==============================] - 4s 821us/step - loss: 2.2662 - categorical_accuracy: 0.1519 - val_loss: 2.2139 - val_categorical_accuracy: 0.1458
Epoch 2/40
4800/4800 [==============================] - 2s 512us/step - loss: 2.1847 - categorical_accuracy: 0.1790 - val_loss: 2.0948 - val_categorical_accuracy: 0.2183
Epoch 3/40
4800/4800 [==============================] - 2s 502us/step - loss: 1.9920 - categorical_accuracy: 0.2590 - val_loss: 1.7763 - val_categorical_accuracy: 0.4225
Epoch 4/40
4800/4800 [==============================] - 2s 499us/step - loss: 1.6808 - categorical_accuracy: 0.3971 - val_loss: 1.3763 - val_categorical_accuracy: 0.5533
Epoch 5/40
4800/4800 [==============================] - 2s 508us/step - loss: 1.4076 - categorical_accuracy: 0.5127 - val_loss: 1.2338 - val_categorical_accuracy: 0.5808
Epoch 6/40
4800/4800 [==========

Epoch 14/40
4800/4800 [==============================] - 2s 514us/step - loss: 0.3932 - categorical_accuracy: 0.8871 - val_loss: 0.3704 - val_categorical_accuracy: 0.8850
Epoch 15/40
4800/4800 [==============================] - 2s 501us/step - loss: 0.3339 - categorical_accuracy: 0.9065 - val_loss: 0.3314 - val_categorical_accuracy: 0.9033
Epoch 16/40
4800/4800 [==============================] - 2s 501us/step - loss: 0.2952 - categorical_accuracy: 0.9196 - val_loss: 0.3472 - val_categorical_accuracy: 0.8992
Epoch 17/40
4800/4800 [==============================] - 2s 503us/step - loss: 0.2596 - categorical_accuracy: 0.9342 - val_loss: 0.3057 - val_categorical_accuracy: 0.9163
Epoch 18/40
4800/4800 [==============================] - 2s 501us/step - loss: 0.2317 - categorical_accuracy: 0.9352 - val_loss: 0.3096 - val_categorical_accuracy: 0.9175
Epoch 19/40
4800/4800 [==============================] - 2s 498us/step - loss: 0.2104 - categorical_accuracy: 0.9446 - val_loss: 0.3065 - val_cat

[94.333333333333343, 94.25, 93.25]